#### INST326 OOP Project 04

Rename this notebook, replacing "_Assignment" with "_YourName"<br>
Insert Signature Block Here

#### Christian Sorensen
> INST326
> Project 04
> 11/25/2024
#### Honor Pledge
> I pledge that the work contained in this assignment is my own, and that I have complied with University and course policies on academic integrity and AI use.


You may work as an individual on **ONE** of the following projects, **OR** if you want to work as a group, contact Dr. Dempwolf for a project assignment. That group assignment will be part of an ongoing research project analyzing innovation ecosystems. 

### Individual Projects
Choose **ONE** of the following projects and write to code solution in the code cell below your choice. Use comments in your code to document your solution. If you need to write comments to the grader, add a markdown cell immediately above your code solution and add your comments there. Be sure to read and follow the Notebook Instructions at the bottom of this notebook. Your grade may depend on it! 

#### 1. Library Management System
>  Objective: Develop a system to manage a library’s collection of books, users, and loan records. This system should allow users to borrow and return books, as well as track which books are currently available.
>
> Requirements
>>- Use classes to represent books, users, and the library.
>>- Implement encapsulation to protect class attributes.
>>- Use inheritance to handle different types of users (e.g., students and teachers).
>>- Demonstrate polymorphism in borrowing rules (e.g., different borrowing limits for students vs. teachers).
>>- Include methods for adding/removing books, registering users, and managing book loans.
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below
import datetime as dt
from datetime import datetime
from __future__ import annotations
from abc import ABC



class Book:
    """
    Book objects are used as 'templates', and hold information about a book in the library catalogue.
    Physical copies of books are tracked using LibraryBook objects, since attributes of the Book iteslf are likely to be identical between them.

    Attributes:
        book_id: Unique identification string for the book, consisting of an 8 digit number. Used to distinguish books with potentially identical titles.
        title (Optional): Title of book. Defaults to "Unknown". Title should basically always be provided
        author (Optional): Author of book. Defaults to "Unknown"
        year (Optional): Year of publication. Defaults to "Unknown"
    """
    def __init__(self, book_id: int, title: str = "Unknown", author: str = "Unknown", year: str = "Unknown"):
        self.book_id = book_id
        self.title = title
        self.author = author
        self.year = year

    def __repr__(self):
        return self.title

    @property
    def book_id(self) -> str:
        return self.__book_id
    
    @book_id.setter
    def book_id(self, new_id: int):
        if type(new_id) is int and new_id >= 0: # Ensure new id is a non-negative integer
            new_id = f"{new_id:3f}"
            self.__book_id = new_id
        else:
            raise ValueError("Id must be non-negative integer")



class LibraryBook:
    """
    LibraryBook objects track individual copies of books in the library system.

    Attributes:
        book_id: The id of the Book that this LibraryBook is a copy of
        sub_id: Unique identifier for this copy of the book
        datetime_entered: Datetime of when this entry was made
    """
    def __init__(self, book_id: str, sub_id: int, datetime_entered: datetime = datetime.now()):

        

class User(ABC):
    """
    Base User abstract class. Subclasses can implement restrictions on loaning books.

    Attributes:
        user_id: Unique identification string for the user. Used to distinguish users with potentially identical names
        firstname: User's first name
        lastname: User's surname
        __balance: Current balance of overdue fees. New books cannot be loaned while balance > 0
    """
    def __init__(self, user_id: int, firstname: str, lastname: str):
        self.user_id = user_id
        self.fname = firstname
        self.lname = lastname
        self.__balance = 0 

    def __repr__(self):
        """Object is represented by lastname, firstname"""
        return f"{self.lname}, {self.fname}"

    @property
    def user_id(self) -> str:
        return self.__user_id
    
    @user_id.setter
    def user_id(self, new_id: int):
        """Ensure new id is a non-negative integer, and stores it as a formatted string"""
        if type(new_id) is int and new_id >= 0:
            new_id = f"{new_id}"
            self.__user_id = new_id
        else:
            raise ValueError("Id must be non-negative integer")
        
    @property
    def balance(self) -> float:
        return self.__balance
    
    def modify_balance(self, amount: int):
        """Modifies the user's current balance by the given amount"""
        if type(amount) in {int, float}:
            self.__balance += amount
            self.__balance = round(self.__balance, 2) # Round to 2 decimal places
        else:
            raise TypeError("Amount must be a float or int")



class Student(User):
    """
    Student class. Has a loan limit of 10

    Attributes:
        user_id: Unique identification string for the user. Used to distinguish users with potentially identical names
        firstname: User's first name
        lastname: User's surname
        __balance: Current balance of overdue fees. New books cannot be loaned while balance > 0
        loan_limit: Number of books the user is allowed to have loaned out at a given time
    """
    def __init__(self, user_id: int, firstname: str, lastname: str):
        super().__init__(user_id, firstname, lastname)
        self.loan_limit = 10



class Teacher(User):
    """
    Teacher class. Has a loan limit of 20

    Attributes:
        user_id: Unique identification string for the user. Used to distinguish users with potentially identical names
        firstname: User's first name
        lastname: User's surname
        __balance: Current balance of overdue fees. New books cannot be loaned while balance > 0
        loan_limit: Number of books the user is allowed to have loaned out at a given time
    """
    def __init__(self, user_id: int, firstname: str, lastname: str):
        super().__init__(user_id, firstname, lastname)
        self.loan_limit = 10



class Loan:
    """
    Tracks information about loaned books for easy management
    
    Attributes:
        full_book_id: Which book was loaned out. Format should be "<Book_id>-<LibraryBook_sub_id>"
        user_id: Which user loaned the book
        datetime_from: Date and time the loan was made
        datetime_due: Date and time the loan is due
    """
    def __init__(self, full_book_id: str, user_id: str, datetime_from: datetime, datetime_due: datetime):
        pass







if __name__ == "__main__":
    book1 = Book()


#### 2. Online Shopping Cart System
>  Objective: Build a shopping cart system for an online store that manages products, shopping carts, and orders.
>
> Requirements
>>- Use classes to represent products, shopping carts, and orders.
>>- Implement encapsulation to handle product prices and cart contents securely.
>>- Use inheritance to create different types of products (e.g., electronics, clothing).
>>- Demonstrate polymorphism by calculating discounts based on product type.
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


#### 3. Restaurant Reservation System
>  Objective: Create a reservation system for a restaurant that manages tables, reservations, and customers.
>
>  Requirements
>>- Use classes to represent tables, customers, and reservations.
>>- Implement encapsulation for managing table availability and reservation details.
>>- Use inheritance to differentiate between walk-in and advance reservations.
>>- Demonstrate polymorphism by handling special cases (e.g., priority seating for VIP customers).
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


#### 4. Vehicle Rental System
>  Objective: Develop a vehicle rental system that manages a fleet of vehicles, customer rentals, and payment processing.
>
>  Requirements
>>- Use classes to represent different types of vehicles, customers, and rental transactions.
>>- Implement encapsulation to handle sensitive information like customer payment details.
>>- Use inheritance to differentiate between various vehicle types (e.g., cars, trucks, motorcycles).
>>- Demonstrate polymorphism by applying different rental pricing strategies based on vehicle type.
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


#### 5. Online Learning Platform
>  Objective: Create an online learning platform that manages courses, students, and instructors.
>  
>  Requirements
>>- Use classes to represent courses, students, and instructors.
>>- Implement encapsulation to manage sensitive information like student grades.
>>- Use inheritance to handle different types of courses (e.g., free, paid, and premium).
>>- Demonstrate polymorphism in applying different grading schemes for assignments.
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


#### 6. E-Commerce Order Processing System
>  Objective: Build an order processing system for an online store that manages products, customers, and orders.
>  
>  Requirements
>>- Use classes to represent products, customers, and orders.
>>- Implement encapsulation for handling payment details securely.
>>- Use inheritance for different types of products (e.g., physical goods, digital downloads).
>>- Demonstrate polymorphism by applying different shipping costs based on product type.
>>- Include execution code to demonstrate that your solution works

In [ ]:
# Solution - enter your code solution below


### Notebook Instructions
> Before turning in your notebook:
> 1. Make sure you have renamed the notebook file as instructed
> 2. Make sure you have included your signature block and that it is correct according to the instructions
> 3. comment your code as necessary
> 4. run all code cells and double check that they run correctly. Include you execution code in your submission. If you can't get your code to run correctly and you want partial credit, add a note for the grader in a new markdown cell directly above your code solution.<br><br>
Turn in your notebook by uploading it to ELMS<br>
IF the exercises involve saved data files, put your notebook and the data file(s) in a zip folder and upload the zip folder to ELMS